In [ ]:
import sys
sys.path.append("..")
import numpy as np
import pandas as pd
import pickle
import functions.calibration as cal
from functions.estimators import Hagan
from functions.estimators import NeuralNetwork
from functions.estimators import Antonov
from py_vollib.black.greeks.analytical import delta
from py_vollib.black.greeks.analytical import vega

### Load Data

In [ ]:
# Import calibrated datasets and Monte Carlo generated hedges
with open("mapping and calibration/files/calibrated_chunks_hagan.pkl", "rb") as f:
    chunks_hagan = pickle.load(f)
with open("mapping and calibration/files/calibrated_chunks_antonov.pkl", "rb") as f:
    chunks_antonov = pickle.load(f)
with open("mapping and calibration/files/calibrated_chunks_nn.pkl", "rb") as f:
    chunks_nn = pickle.load(f)
with open("data generation/test pickles/recomputed_dataset_hedging.pkl", "rb") as f:
    chunks_mc = pickle.load(f)

In [ ]:
# Extract parameters from dictionary
for approximator, chunks in {"hagan": chunks_hagan, "antonov": chunks_antonov, "nn": chunks_nn}.items():
    for _, df in enumerate(chunks):
        df[f"alpha_{approximator}"] = df[f"params_{approximator}"].apply(lambda x: x["alpha"])
        df[f"rho_{approximator}"] = df[f"params_{approximator}"].apply(lambda x: x["rho"])
        df[f"nu_{approximator}"] = df[f"params_{approximator}"].apply(lambda x: x["v"])

### Combine Datasets

In [ ]:
combined_chunks = []

# Loop through each corresponding set of chunks
for i, (df_hagan, df_antonov, df_nn) in enumerate(zip(chunks_hagan, chunks_antonov, chunks_nn)):

    # Combine dataframes and remove duplicates
    combined_df = pd.concat([df_hagan, df_antonov, df_nn], axis=1)
    combined_df = combined_df.loc[:, ~combined_df.columns.duplicated()]  # drop duplicate columns
    combined_chunks.append(combined_df)


In [ ]:
# Merge Monte Carlo generated hedging datasets with calibrated datasets of the different approximator

tol=1e-6

merged_chunks = []

for i, mc_chunk in enumerate(chunks_mc):
    # Extract the reference parameter set
    alpha_ref, nu_ref, rho_ref = mc_chunk[['alpha', 'nu', 'rho']].iloc[0].values

    # Try to find a matching chunk in combined_chunks
    match_idx = None
    for j, comb_chunk in enumerate(combined_chunks):
        a, n, r = comb_chunk[['alpha', 'nu', 'rho']].iloc[0].values
        if (abs(a - alpha_ref) <= tol and 
            abs(n - nu_ref) <= tol and 
            abs(r - rho_ref) <= tol):
            match_idx = j
            break
    
    # Skip if no match found
    if match_idx is None:
        continue
    
    matched_chunk = combined_chunks[match_idx]

    # Keep only rows with common strike prices
    common_strikes = mc_chunk['strike_price'].isin(matched_chunk['strike_price'])
    mc_chunk_filtered = mc_chunk[common_strikes].copy()

    # Identify new columns
    new_cols = [c for c in matched_chunk.columns if c not in mc_chunk_filtered.columns or c == 'strike_price']

    # Merge only on common strike prices
    merged = pd.merge(
        mc_chunk_filtered,
        matched_chunk[new_cols],
        on='strike_price',
        how='left'
    )


    # Add to result
    merged_chunks.append(merged)
    

### Hagan Hedge

In [ ]:
# Calculate the derivatives with regards to f, alpha, rho, and nu of the implied volatilities using central differences approximation
processed_hagan, failed_hagan = cal.IV_derivative(merged_chunks, Hagan, "hagan")

### Neural Network Hedge

In [ ]:
# Calculate the derivatives with regards to f, alpha, rho, and nu of the implied volatilities using central differences approximation
processed_nn, failed_nn = cal.IV_derivative(merged_chunks, NeuralNetwork, "nn")

### ZC Map Hedge

In [ ]:
# Calculate the derivatives with regards to f, alpha, rho, and nu of the implied volatilities using central differences approximation
processed_antonov, failed_antonov = cal.IV_derivative(merged_chunks, Antonov, "antonov")

In [ ]:
# Save dataframes with hedges
with open("hedged_chunks.pkl", "wb") as f:
    pickle.dump(merged_chunks, f)